<a href="https://colab.research.google.com/github/kkt86/quant-notebooks/blob/master/DevNet_Census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DevNet 

Notebook, reproducing the results from Deep Anomaly Detection with Deviation Networks paper (https://paperswithcode.com/paper/deep-anomaly-detection-with-deviation)

Model is applied on the Census dataset, available on https://archive.ics.uci.edu/ml/datasets/Census-Income+(KDD)

In [1]:
%pip install tensorflow==1.10.1 keras==2.2.2

     |████████████████████████████████| 58.4MB 52kB/s 
     |████████████████████████████████| 307kB 36.1MB/s 
     |████████████████████████████████| 3.3MB 34.1MB/s 
     |████████████████████████████████| 573kB 32.9MB/s 
     |████████████████████████████████| 12.2MB 256kB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tifffile 2020.7.24 has requirement numpy>=1.15.1, but you'll have numpy 1.14.5 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: numba 0.48.0 has requirem

In [1]:
import tensorflow as tf
import keras

print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Tensorflow version: 1.10.1
Keras version: 2.2.2


Using TensorFlow backend.


### Load census data

In [2]:
%%bash
tar -xzvf drive/My\ Drive/colab\ data/census.tar.gz

census-income.data
census-income.names
census-income.test


In [3]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector

# load training and testing data
cols = [f"X{i}" for i in range(41)] + ["target"] 
train_data = pd.read_csv("census-income.data", header=None, names=cols)
test_data = pd.read_csv("census-income.test", header=None, names=cols)

# extract numerical and categorical columns
num_cols, cat_cols = [], []
for col in train_data.columns[:-1]:    
  if pd.api.types.is_numeric_dtype(train_data[col]):
    num_cols.append(col)
  else:
    cat_cols.append(col)

  
# create insample and outsample data
insample_X = train_data[num_cols + cat_cols]
insample_y = np.where(train_data["target"] == " - 50000.", 0, 1)
outsample_X = test_data[num_cols + cat_cols]
outsample_y = np.where(test_data["target"] == " - 50000.", 0, 1)

# hotencode categorical values and scale data
transformer = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(), 
     make_column_selector(dtype_include=object)))

insample_X = transformer.fit_transform(insample_X).todense()
outsample_X = transformer.transform(outsample_X).todense()



print(f"Train data shape: {insample_X.shape}")
print(f"Test data shape: {outsample_X.shape}")
print(f"% positive values in train: {sum(insample_y)/len(insample_y)}")
print(f"% positive values in test: {sum(outsample_y)/len(outsample_y)}")

Train data shape: (199523, 409)
Test data shape: (99762, 409)
% positive values in train: 0.06205800834991455
% positive values in test: 0.06200757803572503


### Create baseline model (RF classifier)

In [4]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def print_metrics(y_true, y_pred):
  print(f"Precision : {precision_score(y_true, y_pred)}")
  print(f"Recall    : {recall_score(y_true, y_pred)}")
  print(f"F1-score  : {f1_score(y_true, y_pred)}")
  print("Confusion Matrix: ")
  print(confusion_matrix(y_true, y_pred))  

In [23]:
from sklearn.ensemble import RandomForestClassifier

known_outliers = 100
outlier_indices = np.where(insample_y == 1)[0]
n_outliers = len(outlier_indices)
if n_outliers > known_outliers:
    mn = n_outliers - known_outliers
    remove_idx = rng.choice(outlier_indices, mn, replace=False)
    rf_train_X = np.delete(insample_X, remove_idx, axis=0)
    rf_train_y = np.delete(insample_y, remove_idx, axis=0)

rf = RandomForestClassifier()
rf.fit(rf_train_X, rf_train_y)
outsample_rf_pred = rf.predict(outsample_X)
print_metrics(outsample_y, outsample_rf_pred)

Precision : 0.0
Recall    : 0.0
F1-score  : 0.0
Confusion Matrix: 
[[93576     0]
 [ 6186     0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### DevNet

In [24]:
from keras import regularizers
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, TensorBoard

def dev_network_linear(input_shape):
  '''
  network architecture with no hidden layer, equivalent to linear mapping from
  raw inputs to anomaly scores
  '''    
  x_input = Input(shape=input_shape)
  intermediate = Dense(1, activation='linear',  name = 'score')(x_input)
  return Model(x_input, intermediate)

def dev_network_one(input_shape):
  """
  network architecture with one hidden layer
  """
  x_input = Input(shape=input_shape)
  intermediate = Dense(20, activation="relu", kernel_regularizer=regularizers.l2(0.01), name="hl1")(x_input)
  intermediate = Dense(1, activation="linear", name="score")(intermediate)
  return Model(x_input, intermediate)

def dev_network_two(input_shape):
  """
  network architecture with two hidden layers
  """
  x_input = Input(shape=input_shape)
  intermediate = Dense(250, activation="relu", kernel_regularizer=regularizers.l2(0.01), name="hl1")(x_input)
  intermediate = Dense(20, activation="relu", kernel_regularizer=regularizers.l2(0.03), name="hl2")(intermediate)
  intermediate = Dense(1, activation="linear", name="score")(intermediate)
  return Model(x_input, intermediate)

def dev_network_three(input_shape):
  """
  network architecture with three hidden layers
  """
  x_input = Input(shape=input_shape)
  intermediate = Dense(1000, activation="relu", kernel_regularizer=regularizers.l2(0.01), name="hl1")(x_input)
  intermediate = Dense(250, activation="relu", kernel_regularizer=regularizers.l2(0.01), name="hl2")(intermediate)
  intermediate = Dense(20, activation="relu", kernel_regularizer=regularizers.l2(0.03), name="hl3")(intermediate)
  intermediate = Dense(1, activation="linear", name="score")(intermediate)
  return Model(x_input, intermediate)

def deviation_loss(y_true, y_pred):
  '''
  z-score-based deviation loss
  '''    
  confidence_margin = 5.     
  ## size=5000 is the setting of l in algorithm 1 in the paper
  ref = K.variable(np.random.normal(loc = 0., scale= 1.0, size = 5000) , dtype='float32')
  dev = (y_pred - K.mean(ref)) / K.std(ref)
  inlier_loss = K.abs(dev) 
  outlier_loss = K.abs(K.maximum(confidence_margin - dev, 0.))
  return K.mean((1 - y_true) * inlier_loss + y_true * outlier_loss)

def deviation_network(input_shape, network_depth):
  '''
  construct the deviation network-based detection model
  '''
  if network_depth == 4:
      model = dev_network_three(input_shape)
  elif network_depth == 3:
      model = dev_network_two(input_shape)
  elif network_depth == 2:
      model = dev_network_one(input_shape)
  elif network_depth == 1:
      model = dev_network_linear(input_shape)
  else:
      sys.exit("The network depth is not set properly")
  rms = RMSprop(clipnorm=1.)
  model.compile(loss=deviation_loss, optimizer=rms)
  return model

In [25]:
def batch_generator_sup(x, outlier_indices, inlier_indices, batch_size, nb_batch, rng):
  """
  batch generator
  """
  rng = np.random.RandomState(rng.randint(MAX_INT, size = 1))
  counter = 0
  while 1:                
    ref, training_labels = input_batch_generation_sup(x, outlier_indices, inlier_indices, batch_size, rng)
    counter += 1
    yield(ref, training_labels)
    if (counter > nb_batch):
        counter = 0
 
def input_batch_generation_sup(x_train, outlier_indices, inlier_indices, batch_size, rng):
    '''
    batchs of samples. Alternates between positive and negative pairs.
    '''      
    dim = x_train.shape[1]
    ref = np.empty((batch_size, dim))    
    training_labels = []
    n_inliers = len(inlier_indices)
    n_outliers = len(outlier_indices)
    for i in range(batch_size):    
      if(i % 2 == 0):
        sid = rng.choice(n_inliers, 1)
        ref[i] = x_train[inlier_indices[sid]]
        training_labels += [0.]
      else:
        sid = rng.choice(n_outliers, 1)
        ref[i] = x_train[outlier_indices[sid]]
        training_labels += [1.]
    return np.array(ref), np.array(training_labels)

In [26]:
from sklearn.metrics import average_precision_score, roc_auc_score

def aucPerformance(mse, labels):
  roc_auc = roc_auc_score(labels, mse)
  ap = average_precision_score(labels, mse)
  print("AUC-ROC: %.4f, AUC-PR: %.4f" % (roc_auc, ap))
  return roc_auc, ap;

In [27]:
def load_model_weight_predict(model_name, input_shape, network_depth, x_test):
  '''
  load the saved weights to make predictions
  '''
  model = deviation_network(input_shape, network_depth)
  model.load_weights(model_name)
  scoring_network = Model(inputs=model.input, outputs=model.output)    
  scores = scoring_network.predict(x_test)
  return scores

In [28]:
def inject_noise(seed, n_out, random_seed):
  """
  add anomalies to training data to replicate anomaly contaminated data sets.
  we randomly swape 5% features of anomalies to avoid duplicate contaminated anomalies.
  this is for dense data
  """
  rng = np.random.RandomState(random_seed) 
  n_sample, dim = seed.shape
  swap_ratio = 0.05
  n_swap_feat = int(swap_ratio * dim)
  noise = np.empty((n_out, dim))
  for i in np.arange(n_out):
    outlier_idx = rng.choice(n_sample, 2, replace = False)
    o1 = seed[outlier_idx[0]]
    o2 = seed[outlier_idx[1]]
    swap_feats = rng.choice(dim, n_swap_feat, replace = False)
    noise[i] = o1.copy()
    #noise[i, swap_feats] = o2[swap_feats]
    noise[i, swap_feats] = o2[0, swap_feats]
  return noise

In [29]:
%%bash
mkdir model

mkdir: cannot create directory ‘model’: File exists


In [30]:
import time
from sklearn.model_selection import train_test_split

MAX_INT = np.iinfo(np.int32).max

batch_size = 512      # batch size used in SGD
cont_rate = 0.02      # the outlier contamination rate in the training data
epochs = 50           # the number of epochs
known_outliers = 100  # the number of labeled outliers available at hand"
nb_batch = 20         # the number of batches per epoch
network_depth = 2     # the depth of the network architecture
random_seed = 42      # random seed number
runs = 5             # how many times we repeat the experiments to obtain the average performance

outlier_indices = np.where(insample_y == 1)[0]
outliers = insample_X[outlier_indices]  
n_outliers_org = outliers.shape[0] 

rng = np.random.RandomState(random_seed)
rauc = np.zeros(runs)
ap = np.zeros(runs) 

for run in np.arange(runs):
  x_train, x_test, y_train, y_test = train_test_split(insample_X, insample_y, 
                                                      test_size=0.2, 
                                                      random_state=random_seed, 
                                                      stratify=insample_y)
  y_train = np.array(y_train)
  y_test = np.array(y_test)
  outlier_indices = np.where(y_train == 1)[0]
  inlier_indices = np.where(y_train == 0)[0]
  n_outliers = len(outlier_indices)
  
  n_noise = len(np.where(y_train == 0)[0])*cont_rate / (1. - cont_rate)
  n_noise = int(n_noise)

  rng = np.random.RandomState(random_seed)

  # keep maximum number of outliers below 30 (try removing this)
  if n_outliers > known_outliers:
    mn = n_outliers - known_outliers
    remove_idx = rng.choice(outlier_indices, mn, replace=False)
    x_train = np.delete(x_train, remove_idx, axis=0)
    y_train = np.delete(y_train, remove_idx, axis=0)

  noises = inject_noise(outliers, n_noise, random_seed)
  x_train = np.append(x_train, noises, axis=0)
  y_train = np.append(y_train, np.zeros((noises.shape[0], 1)))

  outlier_indices = np.where(y_train == 1)[0]
  inlier_indices = np.where(y_train == 0)[0]

  input_shape = x_train.shape[1:]
  n_samples_trn = x_train.shape[0]
  n_outliers = len(outlier_indices)            

  start_time = time.time()
  input_shape = x_train.shape[1:]
  model = deviation_network(input_shape, network_depth)
  model_name = "./model/devnet_" + str(cont_rate) + "cr_"  + str(batch_size) +"bs_" + str(known_outliers) + "ko_" + str(network_depth) +"d.h5"
  checkpointer = ModelCheckpoint(model_name, monitor='loss', verbose=0, save_best_only = True, save_weights_only = True)

  model.fit_generator(batch_generator_sup(x_train, outlier_indices, inlier_indices, batch_size, nb_batch, rng), 
                      steps_per_epoch = nb_batch,
                      epochs = epochs,
                      callbacks=[checkpointer], verbose=0) 
  train_time = time.time() - start_time

  start_time = time.time()
  scores = load_model_weight_predict(model_name, input_shape, network_depth, x_test)
  test_time = time.time() - start_time
  rauc[run], ap[run] = aucPerformance(scores, y_test)

AUC-ROC: 0.9100, AUC-PR: 0.5230
AUC-ROC: 0.9116, AUC-PR: 0.5273
AUC-ROC: 0.9112, AUC-PR: 0.5185
AUC-ROC: 0.9131, AUC-PR: 0.5220
AUC-ROC: 0.9156, AUC-PR: 0.5196


In [31]:
outsample_scores = load_model_weight_predict(model_name, input_shape, network_depth, outsample_X)
outsample_pred = np.where(outsample_scores >= 5., 1, 0)
print_metrics(outsample_y, outsample_pred)

Precision : 0.5717740162673115
Recall    : 0.4204655674102813
F1-score  : 0.4845831392640894
Confusion Matrix: 
[[91628  1948]
 [ 3585  2601]]


In [ ]:
[[92843   733]
 [ 3821  2365]]